LangSmith

In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"

In [39]:
import getpass
import os

os.environ["XAI_API_KEY"] = "xai-2wJO6Gsu6Yve6oQ6A7MaAGUIW5FU4yzfUKtHwcGgjVST1WHgOuyTy4gyoxVg1SeOXjpBQEHJyPzT8vOK"

from langchain.chat_models import init_chat_model

model = init_chat_model("grok-3-mini", model_provider="xai", reasoning_effort = "low", tiktoken_model_name="gpt-3.5-turbo")


memory persistent

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage, AIMessage

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# This config contains information that is not part of the input directly, but is still useful. 
# In this case, we want to include a thread_id. This should look like:
config = {"configurable": {"thread_id": "abc123"}}
# This enables us to support multiple conversation threads with a single application

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state


================================== Ai Message ==================================

Hi Bob! I'm Grok, an AI built by xAI. Nice to meet you—what's up? 😊


In [30]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name is Bob when you introduced yourself! If that's changed or if you'd like me to call you something else, just let me know. 😊


prompt template

In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [32]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)
# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}
# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [33]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Jim! I'm Grok, an AI built by xAI. Nice to meet you—what's on your mind? 😊


In [34]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Based on our conversation, you introduced yourself as Jim. If that's not quite right or if you'd like to clarify, just let me know—I'm here to help! 😊 What else can I assist you with?


more complicated prompt

In [35]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

# Define a new graph
workflow = StateGraph(state_schema=State) # state instead of message state
# Define the function that calls the model
def call_model(state: State): # is called whenever the node is invoked
    print("call_model")
    prompt = prompt_template.invoke(state) # Create the prompt from the template and state
    print("Prompt:", prompt)
    response = model.invoke(prompt)
    return {"messages": [response]}
# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [36]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language}, # based on state
    config,
)
output["messages"][-1].pretty_print()

call_model
Prompt: messages=[SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability in Spanish.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='53ebc3ea-dabe-4868-b971-b79c9b18defc')]
================================== Ai Message ==================================

¡Hola, Bob! Soy un asistente útil. ¿En qué puedo ayudarte hoy? 😊


In [37]:
output = app.invoke(
    {"messages": "what is my name", "language": language}, # based on state
    config,
)
output["messages"][-1].pretty_print()

call_model
Prompt: messages=[SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability in Spanish.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='53ebc3ea-dabe-4868-b971-b79c9b18defc'), AIMessage(content='¡Hola, Bob! Soy un asistente útil. ¿En qué puedo ayudarte hoy? 😊', additional_kwargs={'refusal': None, 'reasoning_content': 'First, the user said: "Hi! I\'m Bob." This is a greeting and an introduction.\n\nThe system prompt instructs: "Answer all questions to the best of your ability in Spanish." However, this message isn\'t a question; it\'s a statement. But the prompt says "Answer all questions," which might imply I should respond to interactions in Spanish.\n\nRe-reading the prompt: "Answer all questions to the best of your ability in Spanish." It specifically says "questions," but in the context of being a helpful assistant, I should probably respond to a

Managing Conversation History

In [ ]:
from langchain_core.messages import SystemMessage, AIMessage, trim_messages

# reduce how many messages we're sending to the model.
trimmer = trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [58]:
workflow = StateGraph(state_schema=State)

def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"]) # trim the messages when passing to the model
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [59]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm not sure what your name is! You haven't mentioned it in our conversation so far. If you'd like to share it, I can remember it for future chats. 😊


In [60]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What previous question did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

It looks like this is the first message you've sent in this conversation, so there isn't a previous question from you that I can reference. If you meant to ask about something else or provide more context, feel free to clarify! 😊
